This is a SageMath notebook that attempts to solve xDCP instances: given an elliptic curve $E/\mathbb{F}_q$, an integer $k$ and a polynomial $f\in\mathbb{F}_q[x_1,x_2]$, find $P=(x_1,y_1), Q=(x_2,y_2) \in E(\mathbb{F}_q)$ such that $f(x_1,x_2)=0$ and $k*P = Q$. 


In [2]:
# generate a curve of given size
def gen_setup(bits):
    q = next_prime(2 ** bits)
    F = GF(q)
    for b in F:
        try:
            E = EllipticCurve(F,[-3,b])
        except ArithmeticError:
            pass
        if is_prime(E.order()) and E.order() != q:
            break
    n = E.order()
    return q, E, n

In [25]:
#find l such that both l and k*l are small modulo q
def find_l(k, q):
    M = Matrix(ZZ,2)
    M[0] = [1, k]
    M[1] = [0, q]
    return M.LLL()[0][0]

def make_small(x, K):
    return ZZ(min(K(x), -K(x)))

#construct an auxilliary polynomial as a part of solving xDCP
def construct_poly(E, F, n, l, k, order_multiple=0, reduceby=None):
    R.<x1,x2> = PolynomialRing(F)
    K = GF(n)
    first = E.multiplication_by_m(make_small(l, K))[0](x=x1)
    second = E.multiplication_by_m(ZZ(make_small(l*k, K)) + order_multiple * n)[0](x=x1)
    if order_multiple > 0:
        index = order_multiple * n - make_small(l*k, K)
    else:
        index = make_small(l*k, K)
    if F(index) == 0:
        index += n
#     second = E.multiplication_by_m(index)[0](x=x1)
#     print(first,second)
    
    if reduceby:
        I = R.ideal(reduceby)
        second_num = I.reduce(second.numerator())
        second_den = I.reduce(second.denominator())
        second = second_num / second_den
        poly = (I.reduce(f(x1=first, x2=second).numerator()).univariate_polynomial())(x=x1)
        
    else:
        poly = f(x1=first, x2=second).numerator().univariate_polynomial()
        
    return poly
    
def solve_xdcp(f, E, F, n, k, l, verbose=True):
    
    # catch multiplication_by_0
    assert F(l*k) != 0, "Please choose k,l with nonzero product mod q"
    assert l*k % n != 0, "Please choose k,l with nonzero product mod n"
    
    poly1 = construct_poly(E, F, n, l, k)
    poly2 = construct_poly(E, F, n, l, k, order_multiple=1, reduceby=poly1)
    if k == 1:
        poly = poly1
    else:
        poly = gcd(poly1, poly2)
    
    # if the x-coord is valid, reconstruct the original point
    roots = poly1.roots()
    for r in roots:
        x_coord = r[0]
        if not E.is_x_coord(x_coord):
            continue
        R = E.lift_x(x_coord)
        P = ZZ(l)*R
        Q = ZZ(l*k)*R
        assert f(P[0],Q[0]) == 0, "The found points do not satisfy xdcp"
        if verbose:
            print(f"Points P = {P}, Q = {Q} satisfy xDCP for f = {f}, k = {k}, l = {l}")
        return P, Q
    if verbose:
        print(f"No points P, Q satisfy xDCP for f = {f}, k = {k}")
    return None, None

In [26]:
#test the attack

q, E, n = gen_setup(5)
F = GF(q)
R.<x1,x2> = PolynomialRing(F)
f = x1*x2 + 1
for k in [1..n-1]:
    l = find_l(k, q)
    try:
        solve_xdcp(f, E, F, n, k=k, l=l, verbose=True)
    except:
        pass

Points P = (31 : 25 : 1), Q = (31 : 25 : 1) satisfy xDCP for f = x1*x2 + 1, k = 1, l = 1
Points P = (12 : 6 : 1), Q = (3 : 29 : 1) satisfy xDCP for f = x1*x2 + 1, k = 2, l = 1
Points P = (32 : 26 : 1), Q = (15 : 34 : 1) satisfy xDCP for f = x1*x2 + 1, k = 3, l = 1
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 4
Points P = (36 : 23 : 1), Q = (1 : 9 : 1) satisfy xDCP for f = x1*x2 + 1, k = 5, l = 1
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 6
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 7
Points P = (1 : 28 : 1), Q = (36 : 23 : 1) satisfy xDCP for f = x1*x2 + 1, k = 8, l = -5
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 9
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 10
Points P = (35 : 9 : 1), Q = (19 : 15 : 1) satisfy xDCP for f = x1*x2 + 1, k = 11, l = -3
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 12
No points P, Q satisfy xDCP for f = x1*x2 + 1, k = 13
Points P = (15 : 3 : 1), Q = (32 : 11 : 1) satisfy xDCP for f = x1*x2 + 1, k = 14, l = -3
Point